In [2]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [6]:
Entrez.read(Entrez.einfo())['DbList']

['pubmed', 'protein', 'nuccore', 'nucleotide', 'nucgss', 'nucest', 'structure', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'pubmedhealth', 'seqannot', 'snp', 'sra', 'taxonomy', 'unigene', 'gencoll', 'gtr']

In [8]:
record = Entrez.read(Entrez.egquery(term = 'biopython'))

for row in record['eGQueryResult']:
    if int(row['Count']) > 0:
        print row['DbName'], row['Count']

pubmed 21
pmc 644
books 2
pubmedhealth 2
gds 16
